In [75]:
import pandas as pd

# sedes = pd.read_csv('./sedes_2.csv')
# sedes['Direcci\xc3\xb3n'] = sedes['Direcci\xc3\xb3n'].str[12:]
# sedes['Nombre'] = sedes['Universidad'] +', '+sedes['Facultad'] 

oferta = pd.read_csv('./oferta.csv')

### Cargar datos de oferta academica

In [76]:
ambito = pd.read_csv('./oferta_ambito.csv')
provincia = pd.read_csv('./oferta_provincia.csv')
disciplina = pd.read_csv('./oferta_disciplina.csv')
creacion = pd.read_csv('./facultades_creacion.csv', sep  =';', index_col=0)

In [77]:
# Para las publicas de AMBA hay anio sugerido de comienzo de funcionamiento
creacion['existe_2003'] = creacion.Funcionamiento < 2003
creacion['existe_1988'] = creacion.Funcionamiento < 1988

In [78]:
disciplina.merge(provincia).merge(ambito, how = 'left').merge(creacion[['Universidad', 'Facultad', 'existe_2003', 'existe_1988']], on = ['Universidad', 'Facultad'], how = 'left').to_csv('./oferta_apd.csv', index = False, sep = ';')

In [79]:
sedes = pd.read_csv('./sedes_geoloc_.csv', index_col=0)
oferta = pd.read_csv('./oferta_apd.csv', sep = ';')

estrategicas = pd.read_csv('./estrategicas.csv')

# Marcar carreras estrategicas

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout
import re

# estrategicas['keyword'] = 
# estrategicas['T\xc3\xadtulo'].str.replace('Analista', "")
pd.options.display.max_rows = 999

estrategicas['T\xc3\xadtulo'] = estrategicas['T\xc3\xadtulo'].str.replace(r"\(.*\)","")
pattern = '|'.join(['Analista ', 'Ciencias Básicas ', 'E ', 'El ','Los ', '– ', ', ','De ', 'En ', 'La ', 'Para ', 'Y ',
                    '/a', '/o', 'Licenciatura ', 'Tecnicatura ', 'Universitaria ','Con ', 'Orientada ','Universitario',
                    'Bachiller ', 'Especialista ', 'Especializado ', u'Técnica', u'Técnico', 'Educación', 'Ciclo', 'Inicial', 
                   'con', 'nivel', 'orientación', 'Alternativa', u'Básica', u'Básicas', u'Básico', 'Licenciado',
                   'Profesional', 'Profesor', 'Profesor/a', 'Profesorado', 'Nivel', 'No', 'Para', 'Grado', 'Medio', 'Medios',
                   'Tercer', 'Terciario', 'Recursos', 'Artes', 'Contra'])

estrategicas['T\xc3\xadtulo'] = estrategicas['T\xc3\xadtulo'].str.replace(pattern, '').str.strip().drop_duplicates()
estrategicas_kwd_list = sorted(list(set(' '.join(estrategicas['T\xc3\xadtulo'].dropna().values).split(" ")) - {
            '', 'Básicas', 'Básica', 'Básico', 'Gestión', 'Técnico', 'Educación', 'orientación','Técnica','Orientación', 'Mención', 'ado', 's', 'Del', 'A', 'Las', 'Por'}))

oferta['Estrategica'] = oferta['T\xc3\xadtulo'].str.contains('|'.join(estrategicas_kwd_list))

## Como conviene aniadir Alte Brown?
adicion_UNAB = pd.read_csv('./sedesAlteBrown.csv', sep = ';')
oferta = pd.concat([oferta, adicion_UNAB])

# Corregir campus UTN
oferta.loc[(oferta.Universidad == 'Universidad Tecnol\xc3\xb3gica Nacional') &
          (oferta.Facultad == 'Facultad Regional Buenos Aires'), 'Domicilio'] = \
oferta.loc[(oferta.Universidad == 'Universidad Tecnol\xc3\xb3gica Nacional') &
          (oferta.Facultad == 'Facultad Regional Buenos Aires'), 'Domicilio'].str.replace(
    'Medrano 951 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
    'Mozart 2300 - Ciudad Autonoma de Buenos Aires')

# Save
oferta.to_csv('./oferta_apde.csv', sep = ';', index = False)


### Geolocar

In [53]:
import json
import time
import urllib
import urllib2

def geolocate_rest_api(searchtext, APP_ID = None, APP_CODE = None, BACKOFF_TIME = 30):

    base_url = 'https://geocoder.api.here.com/6.2/geocode.json'

    # This joins the parts of the URL together into one string.
    url = base_url + '?' + urllib.urlencode({
        'searchtext': "%s" % (searchtext),
        'app_id': APP_ID,
        'app_code': APP_CODE,
    })
#     print url
    response = str(urllib2.urlopen(url).read())
    result = json.loads(response.replace('\\n', ''))
    return result


def set_geolocation_values(df):
    if result['Response']['View'] != []:
        core_result = result['Response']['View'][0]['Result'][0]['Location']
        pos = core_result['NavigationPosition']
        address = core_result['Address']

        if 'Country' in address.keys(): df.set_value(i,'Pais', address[u'Country'])
        if 'State' in address.keys(): df.set_value(i,'Provincia', address[u'State'])
        if 'County' in address.keys(): df.set_value(i,'Municipio', address[u'County'])
#         if 'Label' in address.keys(): df.set_value(i,'Direccion', address[u'Label'])
    #     df.set_value(i,'location_id', address[u'LocationId'])
        if 'Longitude' in pos[0].keys(): df.set_value(i,'lng', pos[0]['Longitude'])
        if 'Latitude' in pos[0].keys(): df.set_value(i,'lat', pos[0]['Latitude'])
    else:
        pass

In [89]:
oferta_apde = pd.read_csv('./oferta_apde.csv', sep = ';')
sedes = oferta_apde[['Universidad', 'Facultad', 'Domicilio', 'Provincia']].drop_duplicates()
dataframe = sedes[['Domicilio']].drop_duplicates().reset_index()

col = 'Domicilio'
APP_ID = 'YpwwDaCIpYuxqHCMBAZY'
APP_CODE = 'aI2CPGUAgGi_DwMsTXQK2Q'

for i, row in dataframe.iterrows():
    result = geolocate_rest_api(row[col], APP_ID, APP_CODE)
#     print row[col]
    
    if result:
        set_geolocation_values(dataframe)

    if i%20 == 0:
        print 'processed row '+str(i)

processed row 0
processed row 20
processed row 40
processed row 60
processed row 80
processed row 100
processed row 120
processed row 140
processed row 160
processed row 180
processed row 200
processed row 220
processed row 240
processed row 260
processed row 280
processed row 300
processed row 320
processed row 340
processed row 360
processed row 380
processed row 400
processed row 420
processed row 440
processed row 460
processed row 480
processed row 500
processed row 520
processed row 540
processed row 560
processed row 580
processed row 600
processed row 620
processed row 640


#### Corregir errores detectados

In [90]:
# df = sedes_located.groupby('coords_d').nunique().sort_values(by = ['Universidad', 'Domicilio'], ascending = False)
# #Coords that supposedly have more than one address or more than one university (indicates likely error)
# faulty_coords = df.loc[(df.Domicilio > 2) | (df.Universidad > 1)].index.values
# faulty_coords

#Correct detected mistakes
df = dataframe

# Mendoza
df.loc[(df.lat == -32.89729) & (df.lng == -68.8261), ['lng', 'lat']] = [-68.8119967, -32.8993692]

#Un Cuyo
df.loc[(df.lat == -32.88299) & (df.lng == -68.83742), ['lng', 'lat']] = [-68.876431, -32.881753]

#Un Cat. de Cuyo, San Juan
df.loc[(df.lat == -31.5376072) & (df.lng == -68.5482902), ['lng', 'lat']] = [-68.5885474, -31.5392657]

#Salta
df.loc[(df.lat == -24.719410) & (df.lng == -65.409700), ['lng', 'lat']] = [-24.7399601, -65.3940388]

#UAI Rosario
df.loc[(df.lat == -33.031690) & (df.lng == -60.637283), ['lng', 'lat']] = [-32.955366, -60.653052]

# La Plata
df.loc[df.Domicilio == 'Calle 60 esq. 119 - - La Plata - Buenos Aires ', ['lng', 'lat']] = [-57.9311966, -34.9107029]
df.loc[df.Domicilio == 'Calle 60 y 120 - - La Plata - Buenos Aires ', ['lng', 'lat']] = [-57.9305744, -34.9097438]
df.loc[df.Domicilio == 'Calle 60 esq. 124 S/N - La Plata - Buenos Aires ', ['lng', 'lat']] = [-57.9281661, -34.9047511]

#Buenos Aires

df.loc[df.Domicilio == 'Av. Figueroa Alcorta 7350 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.5478312,-58.4489132]
df.loc[df.Domicilio == 'Federico Lacroze 1947 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.5657843,-58.443185]
df.loc[df.Domicilio == 'Federico Lacroze 1955 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.5657843,-58.443185]
df.loc[df.Domicilio == 'Tte. Gral. Juan D. Per\xc3\xb3n 1818 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.6068853,-58.394589]
df.loc[df.Domicilio == 'Av.Pte.F.Alcorta 2263 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.5828209,-58.394269]
df.loc[df.Domicilio == 'Figueroa Alcorta 7597 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.5450779,-58.45097]
df.loc[df.Domicilio == 'Entre R\xc3\xados 495 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.614806,-58.393997]
df.loc[df.Domicilio == 'Juan Domingo Per\xc3\xb3n 3175 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.6077278,-58.4136839]
df.loc[df.Domicilio == 'Rivadavia 2258 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ', 
       ['lat', 'lng']] = [-34.609725,-58.4006]
df.loc[df.Domicilio == 'C\xc3\xb3rdoba 374 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.5987162,-58.3744635]
df.loc[df.Domicilio == 'Paseo Col\xc3\xb3n 850 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.6175111,-58.3700019]
df.loc[df.Domicilio == 'Las Heras 2191 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.588505,-58.397719]
df.loc[df.Domicilio == 'Antartida Argentina 1535 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.582313,-58.373947]

df.loc[df.Domicilio == 'Florencio Varela 1903 - Villa Luzuriaga - Buenos Aires ',
       ['lat', 'lng']] = [-34.6698856,-58.5647993]
df.loc[df.Domicilio == 'Bme.Mitre 1411 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.6075986,-58.3889115]
df.loc[df.Domicilio == 'Venezuela 925/31 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.6140596,-58.3816285]
df.loc[df.Domicilio == 'Av.Alicia Moreau de 1500 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.6155383,-58.3677917]
df.loc[df.Domicilio == 'Bme. Mitre 1891 - Moreno - Buenos Aires ',
       ['lat', 'lng']] = [-34.6511892,-58.7801321]
df.loc[df.Domicilio == 'Hipólito Irigoyen 3238 - Ciudad Autonoma de Buenos Aires - Ciudad Autonoma de Buenos Aires ',
       ['lat', 'lng']] = [-34.6121836,-58.4144971]
df.loc[df.Domicilio == '6 Nro 777 - La Plata - Buenos Aires ',
       ['lat', 'lng']] = [-34.912504,-57.95283]
df.loc[df.Domicilio == '11 e/ 51 y 53 972 - La Plata - Buenos Aires ',
       ['lat', 'lng']] = [-34.9193462,-57.9546788]
df.loc[df.Domicilio == '51 entre 123 y 124 - - Ensenada - Buenos Aires ',
       ['lat', 'lng']] = [-34.900614,-57.9345815]

#reassign back to geolocation results
dataframe = df

In [97]:
#Detect geolocations with mistaken province, assign handmade corrections

merged = sedes.merge(dataframe, on = 'Domicilio', how = 'left').fillna('')


In [99]:

import unidecode

for col in ['Provincia_x', 'Provincia_y']:
    merged[col] = [unidecode.unidecode(acc).title() for acc in merged[col].str.decode('utf8')]
    
corrections = pd.read_csv('./sedes_geoloc_corrections.csv')
corrections_df = pd.DataFrame(corrections)

for col in ['Provincia_y']:
    df = corrections_df
    df[col] = [unidecode.unidecode(acc).title() for acc in df[col].str.decode('utf8')]


mistakes = merged.loc[merged.Provincia_x != merged.Provincia_y].drop(['Municipio', 'Provincia_y', 'lat', 'lng'], axis = 1)
mistakes_corrected = mistakes.merge(corrections_df, on = ['Universidad', 'Facultad', 'Domicilio'])
mistakes_corrected['Pais'] = 'ARG'

# merged = merged.loc[merged.Provincia_x == merged.Provincia_y]
sedes_located = pd.concat([merged.loc[merged.Provincia_x == merged.Provincia_y], 
                           mistakes_corrected]).drop('Provincia_y', axis = 1).rename(columns = {'Provincia_x': 'Provincia'})

sedes_located.to_csv('sedes_loc_clean.csv', index = False)

In [100]:
df = pd.read_csv('sedes_loc_clean.csv')

### Agrupar en 'Zonas Universitarias' (solo publicas AMBA)

In [101]:
def clip_lat_lng(df, coords):
    """
    coords: ((lng_o, lng_e) ,(lat_s, lat_n))
    """
    ((O, E), (S, N)) = coords
    
    df_clip = df.loc[(df['lng'] > O) & (df['lng'] <= E) & (df['lat'] < N) & (df['lat'] >= S)]
    return df_clip

sedes_located = pd.read_csv('sedes_loc_clean.csv')
#clip AMBA
coords_AMBA = ((-59.2, -57.8), (-35.1, -34.05))
sedes_AMBA = clip_lat_lng(sedes_located, coords_AMBA)
#Only public
oferta_apde = pd.read_csv('./oferta_apde.csv', sep = ';')
sedes_AMBA = sedes_AMBA.merge(oferta_apde[['Universidad', 'Ámbito']].drop_duplicates())
sedes_AMBA = sedes_AMBA.loc[sedes_AMBA['Ámbito'] == 'Público']
#coords column for distances
sedes_AMBA['coords_d'] = list(zip(sedes_AMBA.lat, sedes_AMBA.lng))

In [35]:
sedes_AMBA[['Universidad', 'Facultad', 'Domicilio']].sort_values(by = ['Universidad', 'Facultad']).drop_duplicates().to_csv('./creacion_facultades.csv', sep = ';')

In [34]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# clip_lat_lng(oferta_apde.loc[oferta_apde['Ámbito'] == 'Público'], coords_AMBA)
sedes_AMBA.merge(oferta_apde).sort_values(by = ['Universidad', 'Facultad'])

,Domicilio,Facultad,Municipio,Pais,Provincia,Universidad,index,lat,lng,Ámbito,coords_d,Título,Tipo de Título,Duración,Condiciones de Ingreso,Teléfono,Web,Mail,Disciplina,Estrategica
713,Bulnes esquina Guiraldes - Merlo - Buenos Aires,Escuela de Electrónica de Defensa,Merlo,ARG,Buenos Aires,Instituto Universitario Aeronáutico,5397,-34.688340,-58.760700,Público,"(-34.6883399, -58.7607)",Técnico Universitario en Vigilancia y Control ...,Otros Pregrados,2 Años,Consulte con la Institución,(00220) 4821181,http://www.iua.edu.ar/,uaa_iua_merlo@iua.edu.ar,Otras Ciencias Aplicadas,True
636,Rio Santiago - Base Naval De Rio Santiago - Bu...,Unidad Académica Escuela Naval Militar,Ensenada,ARG,Buenos Aires,Instituto Universitario Naval,1941,-34.844960,-57.898148,Público,"(-34.8449602, -57.8981484)",Licenciado en Recursos Navales para la Defensa...,Grado,5 Años,Consulte con la Institución,(00221) 4680330,www.escuelanaval.mil.ar,ingreso@escuelanaval.mil.ar,"Ciencias Políticas, Relaciones Internacionales...",True
637,Rio Santiago - Base Naval De Rio Santiago - Bu...,Unidad Académica Escuela Naval Militar,Ensenada,ARG,Buenos Aires,Instituto Universitario Naval,1941,-34.844960,-57.898148,Público,"(-34.8449602, -57.8981484)",Licenciado en Recursos Navales para la Defensa...,Grado,5 Años,Consulte con la Institución,(00221) 4680330,www.escuelanaval.mil.ar,ingreso@escuelanaval.mil.ar,"Ciencias Políticas, Relaciones Internacionales...",True
638,Rio Santiago - Base Naval De Rio Santiago - Bu...,Unidad Académica Escuela Naval Militar,Ensenada,ARG,Buenos Aires,Instituto Universitario Naval,1941,-34.844960,-57.898148,Público,"(-34.8449602, -57.8981484)",Licenciado en Recursos Navales para la Defensa...,Grado,5 Años,Consulte con la Institución,(00221) 4680330,www.escuelanaval.mil.ar,ingreso@escuelanaval.mil.ar,"Ciencias Políticas, Relaciones Internacionales...",True
639,Rio Santiago - Base Naval De Rio Santiago - Bu...,Unidad Académica Escuela Naval Militar,Ensenada,ARG,Buenos Aires,Instituto Universitario Naval,1941,-34.844960,-57.898148,Público,"(-34.8449602, -57.8981484)",Licenciado en Recursos Navales para la Defensa...,Grado,5 Años,Consulte con la Institución,(00221) 4680330,www.escuelanaval.mil.ar,ingreso@escuelanaval.mil.ar,"Ciencias Políticas, Relaciones Internacionales...",True
640,Rio Santiago - Base Naval De Rio Santiago - Bu...,Unidad Académica Escuela Naval Militar,Ensenada,ARG,Buenos Aires,Instituto Universitario Naval,1941,-34.844960,-57.898148,Público,"(-34.8449602, -57.8981484)",Licenciado en Administración Naval,Grado,60 Meses,Consulte con la Institución,(00221) 4680330,www.escuelanaval.mil.ar,ingreso@escuelanaval.mil.ar,Economía y Administración,True
694,El Quebracho y el Yacaré S/N - Ciudad Evita - ...,Unidad Académica General Don Martín Miguel de ...,La Matanza,ARG,Buenos Aires,Instituto Universitario de Gendarmería Nacional,2263,-34.734920,-58.534570,Público,"(-34.73492, -58.53457)",Analista Universitario en Administración Pública,Título Intermedio,3 Años,Consulte con la Institución,(0011) 46202374,http://www.gendarmeria.gov.ar/,iugna@gendarmeria.gov.ar,Economía y Administración,True
695,El Quebracho y el Yacaré S/N - Ciudad Evita - ...,Unidad Académica General Don Martín Miguel de ...,La Matanza,ARG,Buenos Aires,Instituto Universitario de Gendarmería Nacional,2263,-34.734920,-58.534570,Público,"(-34.73492, -58.53457)",Licenciado en Administración Pública,Grado,4 Años,Consulte con la Institución,(0011) 46202374,http://www.gendarmeria.gov.ar/,iugna@gendarmeria.gov.ar,Economía y Administración,True
696,El Quebracho y el Yacaré S/N - Ciudad Evita - ...,Unidad Académica General Don Martín Miguel de ...,La Matanza,ARG,Buenos Aires,Instituto Universitario de Gendarmería Nacional,2263,-34.734920,-58.534570,Público,"(-34.73492, -58.53457)",Licenciado en Gestión y Seguridad de las Tecno...,Grado,4 Años,Consulte con la Institución,(0011) 46202374,http://www.gendarmeria.gov.ar/,iugna@gendarmeria.gov.ar,Informática,True
697,El Quebracho y el Yac

In [161]:
df = sedes_AMBA
df_for_dist = df['coords_d'].reset_index()
df_for_dist['key'] = 0
df_for_dist.merge(df_for_dist, on = 'key')


,index_x,coords_d_x,key,index_y,coords_d_y
0,0,"(-34.57476, -58.43563)",0,0,"(-34.57476, -58.43563)"
1,0,"(-34.57476, -58.43563)",0,1,"(-34.5721833, -58.4338789)"
2,0,"(-34.57476, -58.43563)",0,2,"(-34.60934, -58.59185)"
3,0,"(-34.57476, -58.43563)",0,47,"(-34.59807, -58.48238)"
4,0,"(-34.57476, -58.43563)",0,48,"(-34.62831, -58.4467)"
5,0,"(-34.57476, -58.43563)",0,49,"(-34.54684, -58.43995)"
6,0,"(-34.57476, -58.43563)",0,50,"(-34.54684, -58.43995)"
7,0,"(-34.57476, -58.43563)",0,51,"(-34.6727, -58.50476)"
8,0,"(-34.57476, -58.43563)",0,52,"(-34.5962099, -58.4775)"
9,0,"(-34.57476, -58.43563)",0,53,"(-34.597903, -58.3974584)"


### Drafts

In [108]:
AMBA_lng = (sedes_located['lng'] > -59.2) & (sedes_located['lng'] <= -57.8)
AMBA_lat = (sedes_located['lat'] < -34.05) & (sedes_located['lat'] >= -35.1)

sedes_AMBA = sedes_located.loc[AMBA_lng & AMBA_lat]

In [113]:
oferta_apde = pd.read_csv('./oferta_apde.csv', sep = ';')
oferta_estrat_AMBA = oferta_apde.loc[(oferta_apde['Ámbito'] == 'Público') & oferta_apde['Estrategica']].merge(sedes_AMBA, on = ['Universidad', 'Facultad', 'Domicilio'], how = 'left').dropna()

In [114]:
pd.set_option('display.max_colwidth', -1)
oferta_estrat_AMBA.groupby('Disciplina')['Título'].unique()

Disciplina
Arqueología                                                    [Licenciado en Paleontología]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 